In [1]:
import aiohttp
import asyncio
import logging

async def get_access_token(tenant_id, client_id, client_secret):
    token_url = f"https://login.microsoftonline.com/{tenant_id}/oauth2/v2.0/token"
    data = {
        'client_id': client_id,
        'scope': 'https://graph.microsoft.com/.default',
        'client_secret': client_secret,
        'grant_type': 'client_credentials'
    }
    async with aiohttp.ClientSession() as session:
        async with session.post(token_url, data=data) as response:
            token_response = await response.json()
            if 'access_token' in token_response:
                return token_response['access_token']
            else:
                raise Exception(f"Error obtaining access token: {token_response}")

def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

async def send_batch_requests(session, requests, headers):
    batch_url = 'https://graph.microsoft.com/v1.0/$batch'
    batch_request_body = {'requests': requests}
    async with session.post(batch_url, headers=headers, json=batch_request_body) as response:
        batch_response = await response.json()
        if 'error' in batch_response:
            raise Exception(f"Error in batch request: {batch_response['error']}")
        return batch_response.get('responses', [])

async def get_all_teams(session, headers):
    url = "https://graph.microsoft.com/v1.0/groups?$filter=resourceProvisioningOptions/Any(x:x eq 'Team')&$select=id,displayName"
    all_teams = []
    while url:
        async with session.get(url, headers=headers) as response:
            data = await response.json()
            if 'error' in data:
                raise Exception(f"Error fetching teams: {data['error']}")
            all_teams.extend(data.get('value', []))
            url = data.get('@odata.nextLink')
    return all_teams

async def get_team_channels_with_files(session, headers, team_id):
    channels = []
    url = f"https://graph.microsoft.com/v1.0/teams/{team_id}/channels?$select=id,displayName"
    while url:
        async with session.get(url, headers=headers) as response:
            data = await response.json()
            if 'error' in data:
                raise Exception(f"Error fetching channels: {data['error']}")
            channels.extend(data.get('value', []))
            url = data.get('@odata.nextLink')

    if not channels:
        return []

    batch_requests = []
    id_to_channel = {}
    for idx, channel in enumerate(channels):
        request = {
            'id': str(idx),
            'method': 'GET',
            'url': f"/teams/{team_id}/channels/{channel['id']}/filesFolder?$select=id,name,parentReference"
        }
        batch_requests.append(request)
        id_to_channel[str(idx)] = channel

    channel_results = []
    for batch in chunks(batch_requests, 20):
        batch_responses = await send_batch_requests(session, batch, headers)
        for response in batch_responses:
            channel = id_to_channel[response['id']]
            data = response.get('body', {})
            if 'error' in data:
                logging.error(f"Error in batch response for channel {channel['id']}: {data['error']}")
                files_folder = None
            else:
                files_folder = {
                    'id': data.get('id'),
                    'name': data.get('name'),
                    'driveId': data.get('parentReference', {}).get('driveId')
                }
            channel_results.append({
                'id': channel['id'],
                'displayName': channel['displayName'],
                'filesFolder': files_folder
            })
    return channel_results

async def get_just_teams(tenant_id, client_id, client_secret):
    access_token = await get_access_token(tenant_id, client_id, client_secret)
    headers = {'Authorization': f'Bearer {access_token}'}
    async with aiohttp.ClientSession() as session:
        return await get_all_teams(session, headers)

async def get_all_teams_with_channels_and_files(tenant_id, client_id, client_secret):
    access_token = await get_access_token(tenant_id, client_id, client_secret)
    headers = {'Authorization': f'Bearer {access_token}'}
    async with aiohttp.ClientSession() as session:
        all_teams = await get_all_teams(session, headers)
        tasks = [get_team_channels_with_files(session, headers, team['id']) for team in all_teams]
        teams_channels = await asyncio.gather(*tasks)
        return [
            {'id': team['id'], 'displayName': team['displayName'], 'channels': channels}
            for team, channels in zip(all_teams, teams_channels)
        ]

async def get_team_channels_with_files_single_team(tenant_id, client_id, client_secret, team_id):
    access_token = await get_access_token(tenant_id, client_id, client_secret)
    headers = {'Authorization': f'Bearer {access_token}'}
    async with aiohttp.ClientSession() as session:
        return await get_team_channels_with_files(session, headers, team_id)


In [2]:
import orjson, os
with open("local.settings.json") as f:
    os.environ.update(orjson.loads(f.read())["Values"])

credential = {
    "tenant_id": os.environ["MSGRAPH_TENANT_ID"],
    "client_id": os.environ["MSGRAPH_CLIENT_ID"],
    "client_secret": os.environ["MSGRAPH_CLIENT_SECRET"]
}

In [3]:
all_data = await get_all_teams_with_channels_and_files(**credential)
display(len(all_data))
display(all_data[30])

237

{'id': '97234c8b-c651-436d-80f9-36e3257f7684',
 'displayName': 'California Closets San Diego',
 'channels': [{'id': '19:4519f15dc9da4c978e39012728d8a409@thread.tacv2',
   'displayName': 'Creatives',
   'filesFolder': {'id': '01LL6BMQWQRN34UV3NL5DYRGDV2HF3V6NW',
    'name': 'Creatives',
    'driveId': 'b!FsnZDEPa_0KOowgcpWsqPqV2YmoZ_zNMnAhNmYBhqgTCJOZ7h4ZySoiyxHG1lyZw'}},
  {'id': '19:0k9InGU1MjEMbVqr431L7CW0wEnW1XknL0Lb8eXnKik1@thread.tacv2',
   'displayName': 'General',
   'filesFolder': {'id': '01WYNLKBFKENZJMGBG65HLSOY2MK2IB5JT',
    'name': 'General',
    'driveId': 'b!ucPH7aGaVUOld_joRVJzbyLCobZLh5xJvpWKol6b1EQQl8R4PW90SalIJhWIJ5Xh'}},
  {'id': '19:817c0e4ed82b43b4b5f76dab5ee92676@thread.tacv2',
   'displayName': 'PMs',
   'filesFolder': {'id': '01SFP3T52UMEK5DWNC5RC3BSIOQ7FHI54F',
    'name': 'PMs',
    'driveId': 'b!dY9bUAM1K02sjLSTpMrnfaV2YmoZ_zNMnAhNmYBhqgTCJOZ7h4ZySoiyxHG1lyZw'}},
  {'id': '19:9743a89aee0449b1a2ec7257b306a17e@thread.tacv2',
   'displayName': 'AdOps',
   'file

In [5]:
all_teams = await get_just_teams(**credential)
all_channels = await get_team_channels_with_files_single_team(**credential, team_id=all_teams[30]["id"])
display(len(all_teams))
display(all_teams[30])
display(all_channels)

237

{'id': '97234c8b-c651-436d-80f9-36e3257f7684',
 'displayName': 'California Closets San Diego'}

[{'id': '19:0k9InGU1MjEMbVqr431L7CW0wEnW1XknL0Lb8eXnKik1@thread.tacv2',
  'displayName': 'General',
  'filesFolder': {'id': '01WYNLKBFKENZJMGBG65HLSOY2MK2IB5JT',
   'name': 'General',
   'driveId': 'b!ucPH7aGaVUOld_joRVJzbyLCobZLh5xJvpWKol6b1EQQl8R4PW90SalIJhWIJ5Xh'}},
 {'id': '19:4519f15dc9da4c978e39012728d8a409@thread.tacv2',
  'displayName': 'Creatives',
  'filesFolder': {'id': '01LL6BMQWQRN34UV3NL5DYRGDV2HF3V6NW',
   'name': 'Creatives',
   'driveId': 'b!FsnZDEPa_0KOowgcpWsqPqV2YmoZ_zNMnAhNmYBhqgTCJOZ7h4ZySoiyxHG1lyZw'}},
 {'id': '19:9743a89aee0449b1a2ec7257b306a17e@thread.tacv2',
  'displayName': 'AdOps',
  'filesFolder': {'id': '01F67SEWA3XGTERPQSAZCJAJHKZNP7WNSM',
   'name': 'AdOps',
   'driveId': 'b!L5zJkbS2DEuHWEZHpacTiKV2YmoZ_zNMnAhNmYBhqgTCJOZ7h4ZySoiyxHG1lyZw'}},
 {'id': '19:817c0e4ed82b43b4b5f76dab5ee92676@thread.tacv2',
  'displayName': 'PMs',
  'filesFolder': {'id': '01SFP3T52UMEK5DWNC5RC3BSIOQ7FHI54F',
   'name': 'PMs',
   'driveId': 'b!dY9bUAM1K02sjLSTpMrnfaV2YmoZ_zNMn